In [1]:
import pandas as pd
import numpy as  np
import warnings
from skimpy import skim
import json
import sqlite3


In [2]:
# connect to skyanalytics_swamp.db
con = sqlite3.connect('skyanalytics_swamp.db')

In [3]:
# create df airplanes
df_airplanes = pd.read_sql_query('SELECT * FROM airplanes', con)
df_airplanes.head()

,ref_aero,type_model,debut_service,last_maint,en_maintenance,end_maint
0,E170_6353,E170,2000-03-17,2024-03-27 00:00:00,False,
1,B767_2803,B767,2020-12-03,2024-06-01 00:00:00,False,None
2,CRJ700_6943,CRJ700,2015-01-29,2024-06-02,True,2024-06-03 14:13:00
3,A350_1177,A350,2005-08-07,2024-01-04 00:00:00,False,
4,B777_4029,B777,2009-01-01,2024-06-01 00:00:00,False,None


In [5]:
df_airplanes[df_airplanes['ref_aero'].isin(df_airplanes['ref_aero'][df_airplanes['ref_aero'].duplicated()])].sort_values("ref_aero")

,ref_aero,type_model,debut_service,last_maint,en_maintenance,end_maint
40,E170_5551,E170,2017-02-07,2024-06-01 00:00:00,False,None
230,E170_5551,E170,2021-04-10,2024-06-01 00:00:00,False,None


In [6]:
df_airplanes.drop_duplicates(subset=['ref_aero'], keep='first', inplace=True)

In [4]:
# create logs_vol
df_logs_vol = pd.read_sql_query('SELECT * FROM logs_vol', con)
df_logs_vol.head()

,ref_vol,aero_linked,jour_vol,time_en_air,sensor_data,etat_voyant
0,V06783026,A320_6242,2024-06-04,6.2,"{'temp': '-14.7°C', 'pressure': '1009.1 hPa', ...",1
1,V06810954,A380_5199,2024-06-04,2.1,"{'temp': '2.9°C', 'pressure': '1015.8 hPa', 'v...",0
2,V05201226,A350_3122,2024-06-04,7.7,"{'temp': '8.0°C', 'pressure': '934.2 hPa', 'vi...",0
3,V05582404,B747_3165,2024-06-04,8.3,"{'temp': '7.3°C', 'pressure': '999.9 hPa', 'vi...",0
4,V01713095,A350_6452,2024-06-04,9.8,"{'temp': '-11.4°C', 'pressure': '946.7 hPa', '...",0


In [8]:
# transform sensor_data into 3 columns
df_logs_vol['sensor_data'] = df_logs_vol['sensor_data'].apply(lambda x: json.loads(x.replace('\'', '\"')))
df_logs_vol_sensor = df_logs_vol.join(pd.json_normalize(df_logs_vol['sensor_data']))
df_logs_vol_sensor.head()

,ref_vol,aero_linked,jour_vol,time_en_air,sensor_data,etat_voyant,temp,pressure,vibrations
0,V06783026,A320_6242,2024-06-04,6.2,"{'temp': '-14.7°C', 'pressure': '1009.1 hPa', ...",1,-14.7°C,1009.1 hPa,0.015339792011828113 m/s²
1,V06810954,A380_5199,2024-06-04,2.1,"{'temp': '2.9°C', 'pressure': '1015.8 hPa', 'v...",0,2.9°C,1015.8 hPa,2.317763888323717 m/s²
2,V05201226,A350_3122,2024-06-04,7.7,"{'temp': '8.0°C', 'pressure': '934.2 hPa', 'vi...",0,8.0°C,934.2 hPa,1.9993783731813313 m/s²
3,V05582404,B747_3165,2024-06-04,8.3,"{'temp': '7.3°C', 'pressure': '999.9 hPa', 'vi...",0,7.3°C,999.9 hPa,0.8753830139272639 m/s²
4,V01713095,A350_6452,2024-06-04,9.8,"{'temp': '-11.4°C', 'pressure': '946.7 hPa', '...",0,-11.4°C,946.7 hPa,1.5515313798653603 m/s²


In [9]:
# combine df_airplanes and df_logs_vol to create clean table
df_combined = pd.concat([df_airplanes, df_logs_vol_sensor], axis=1)
#df_combined.head()
# need more time for reflections. Got no idea right now


,ref_aero,type_model,debut_service,last_maint,en_maintenance,end_maint,ref_vol,aero_linked,jour_vol,time_en_air,sensor_data,etat_voyant,temp,pressure,vibrations
0,E170_6353,E170,2000-03-17,2024-03-27 00:00:00,False,,V06783026,A320_6242,2024-06-04,6.2,"{'temp': '-14.7°C', 'pressure': '1009.1 hPa', ...",1,-14.7°C,1009.1 hPa,0.015339792011828113 m/s²
1,B767_2803,B767,2020-12-03,2024-06-01 00:00:00,False,None,V06810954,A380_5199,2024-06-04,2.1,"{'temp': '2.9°C', 'pressure': '1015.8 hPa', 'v...",0,2.9°C,1015.8 hPa,2.317763888323717 m/s²
2,CRJ700_6943,CRJ700,2015-01-29,2024-06-02,True,2024-06-03 14:13:00,V05201226,A350_3122,2024-06-04,7.7,"{'temp': '8.0°C', 'pressure': '934.2 hPa', 'vi...",0,8.0°C,934.2 hPa,1.9993783731813313 m/s²
3,A350_1177,A350,2005-08-07,2024-01-04 00:00:00,False,,V05582404,B747_3165,2024-06-04,8.3,"{'temp': '7.3°C', 'pressure': '999.9 hPa', 'vi...",0,7.3°C,999.9 hPa,0.8753830139272639 m/s²
4,B777_4029,B777,2009-01-01,2024-06-01 00:00:00,False,None,V01713095,A350_6452,2024-06-04,9.8,"{'temp': '-11.4°C', 'pressure': '946.7 hPa', '...",0,-11.4°C,946.7 hPa,1.5515313798653603 m/s²
